In [1]:
import pandas as pd
import altair as alt
import numpy as np
from altair import datum
import geopandas as gpd
import json
import requests
import pytz
from shapely.ops import nearest_points
from sklearn.neighbors import BallTree
import statsmodels.api as sm
import statsmodels.formula.api as smf
from altair_saver import save
from scipy.special import erf
import scipy.stats as st
from datetime import datetime
from datetime import date

alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

In [2]:

%%html
<style>
@import url('https://fonts.googleapis.com/css?family=Roboto');
</style>

In [3]:
%run easley_dev_theme.py

# Data Import
The first set of data looked at comes from the sources noted below.  For January the data comes from the ERCOT rolling 15 minutes settlements, while for the February the EIA hourly data was used

In [4]:
#EIA data source https://www.eia.gov/beta/electricity/gridmonitor/dashboard/custom/pending
eia_df = pd.read_csv('assets/930-data-export.csv')
#ERCOT data source http://www.ercot.com/content/wcm/lists/181766/IntGenbyFuel2021.xlsx
#ERCOT webpage http://www.ercot.com/gridinfo/generation
ercot_df = pd.read_csv('assets/jan_gen_by_fuel_ercot.csv')

In [5]:
# Uncomment to review EIA data
#eia_df.head()

In [6]:
# Uncomment to review ERCOT data
#ercot_df.head()

The next two cells handle making the data between ERCOT and the EIA consistent with each other, primarly.
1. Standardizing date formats
2. Combining ERCOT gas combined cycle and other data into one gas dataset
3. Adding biomass to other in ERCOT
4. Converting the 15 min ERCOT settlements to Megawatt-Hours

In [7]:
#Formatting clean-up
ercot_df = (ercot_df.drop(columns=['Total'])
            .set_index(['Date','Fuel','Settlement Type'])
            .stack()
            .reset_index()
            .rename(columns={'level_3':'hour',0:'power'})
           )
#Date clean-up
ercot_df['datetime'] = pd.to_datetime(ercot_df['Date'] + ' ' + ercot_df['hour']).dt.tz_localize('America/Chicago')
ercot_df = ercot_df.drop(columns=['Date','hour','Settlement Type'])
#Gas data combination
gas_total = ercot_df[ercot_df['Fuel']=='Gas'].set_index('datetime')
gas_cc_total = ercot_df[ercot_df['Fuel']=='Gas-CC'].set_index('datetime').rename(columns={'Fuel':'Fuel_CC','power':'power_CC'})
gas_total = pd.concat([gas_total, gas_cc_total], axis=1)
gas_total['power'] = gas_total['power']+gas_total['power_CC']
gas_total = gas_total.drop(columns=['Fuel_CC','power_CC'])
gas_total['Fuel'] = 'Gas-Total'
gas_total = gas_total.reset_index()
#Adding biomass to other
other_total = ercot_df[ercot_df['Fuel']=='Other'].set_index('datetime')
bio_total = ercot_df[ercot_df['Fuel']=='Biomass'].set_index('datetime').rename(columns={'Fuel':'Fuel_bio','power':'power_bio'})
other_total = pd.concat([other_total, bio_total], axis=1)
other_total['power'] = other_total['power']+bio_total['power_bio']
other_total = other_total.drop(columns=['Fuel_bio','power_bio'])
other_total['Fuel'] = 'Other-Total'
other_total = other_total.reset_index()
#Convert 15 min settlements to MW-h
ercot_df = pd.concat([ercot_df,gas_total,other_total])
ercot_df['power'] = ercot_df['power']*4
ercot_df.head()

del gas_total,gas_cc_total,other_total,bio_total

In [8]:
#Time format made consistent
central = pytz.timezone('America/Chicago')
eia_df['Timestamp (Hour Ending)'] = eia_df['Timestamp (Hour Ending)'].replace({'CST': 'UTC+6'}, regex=True)
eia_df['datetime']  = pd.to_datetime(eia_df['Timestamp (Hour Ending)'], utc=True).dt.tz_convert(central)
#Formatting and name clean-up
eia_df = (eia_df.drop(columns=['BA Code','Timestamp (Hour Ending)'])
          .set_index(['datetime'])
          .rename(columns={
              'Wind Generation (MWh)':'Wind',
              'Solar Generation (MWh)':'Solar',
              'Hydro Generation (MWh)':'Hydro',
              'Other Generation (MWh)':'Other-Total',
              'Natural gas Generation (MWh)':'Gas-Total',
              'Coal Generation (MWh)':'Coal',
              'Nuclear Generation (MWh)':'Nuclear',
          })
          .stack()
          .reset_index()
          .rename(columns={'level_1':'Fuel',0:'power'})
         )
#eia_df.head(5)

In [9]:
#Give preference to ERCOT data over EIA data and combine the datasets
eia_filtered = eia_df[eia_df['datetime']>ercot_df['datetime'].max()]
ercot_used = ercot_df[~ercot_df['Fuel'].isin(['Biomass','Gas','Other','Gas-CC'])]
combined_df = pd.concat([ercot_used,eia_filtered]).set_index('datetime')

del ercot_used,ercot_df,eia_filtered,eia_df

# Generation Graphing

The next set of cells works on similar visualizations to part one, but this time in probability space

In [10]:
# Hourly z_score and probability calculation
hourly_mean = combined_df.groupby([combined_df.index.hour,'Fuel']).mean().reset_index()
hourly_mean = hourly_mean.rename(columns={'power':'power_mean', 'datetime':'hour'})
hourly_std = combined_df.groupby([combined_df.index.hour,'Fuel']).std().reset_index()
hourly_std = hourly_std.rename(columns={'power':'power_std', 'datetime':'hour'})

l_combined_df = combined_df.copy()
l_combined_df['hour'] = l_combined_df.index.hour
l_combined_df = l_combined_df.reset_index().merge(hourly_mean, on=['hour','Fuel'])
l_combined_df = l_combined_df.reset_index().merge(hourly_std, on=['hour','Fuel'])
l_combined_df = l_combined_df.set_index('datetime')
l_combined_df['prob'] = 0.5 * (1 + erf((l_combined_df['power'] - l_combined_df['power_mean'])/np.sqrt(2 * l_combined_df['power_std']**2)))
l_combined_df['z_score'] = (l_combined_df['power'] - l_combined_df['power_mean'])/l_combined_df['power_std']

#l_combined_df

In [11]:
feb_df = l_combined_df.reset_index()
feb_df = feb_df[feb_df['datetime']>pd.to_datetime('2021-02-01 00:00:00-06:00')]

feb_chart = alt.Chart(feb_df).transform_filter(
    (datum.Fuel=='Gas-Total') | (datum.Fuel=='Wind')
).mark_line().encode(
    x=alt.X('datetime:T',
         axis=alt.Axis(title='Date')
     ),
    y=alt.Y('z_score:Q',
         axis=alt.Axis(title='Probability of Generating Less Than, %', labels=False, grid=False)
     ),
    color=alt.Color('Fuel')
)

wind_problem = alt.Chart({'values':[{'x': '2021-02-08', 'y': 4}]}).mark_text(
    text='Wind Problems Start', angle=270, color='black', 
).encode(
    x='x:T', y='y:Q'
)

near_grid_collapse = alt.Chart({'values':[{'x': '2021-02-14 20:00:00-06:00', 'y': 3}]}).mark_text(
    text='Near Grid Collapse', angle=270, color='black'
).encode(
    x='x:T', y='y:Q'
)

rule_wind = alt.Chart({'values':[{'x': '2021-02-08 00:00:00'}]}).mark_rule(color='gray').encode(
    x='x:T',
)

rule_collapse= alt.Chart({'values':[{'x': '2021-02-15 00:02:00-06:00'}]}).mark_rule(color='gray').encode(
    x='x:T',
)

text_P50 = alt.Chart({'values':[{'x': '2021-02-01', 'y': 0}]}).mark_text(
    text='50', angle=0, color='#B3B3B3'
).encode(
    x='x:T', y='y:Q'
)

rule_P50= alt.Chart({'values':[{'y': 0}]}).mark_rule(color='#B3B3B3').encode(
    y='y:Q',
)

text_P90 = alt.Chart({'values':[{'x': '2021-02-01', 'y': 1.281551566}]}).mark_text(
    text='90', angle=0, color='#B3B3B3'
).encode(
    x='x:T', y='y:Q'
)

rule_P90= alt.Chart({'values':[{'y': 1.281551566}]}).mark_rule(color='#B3B3B3').encode(
    y='y:Q',
)

text_P10 = alt.Chart({'values':[{'x': '2021-02-01', 'y': -1.281551566}]}).mark_text(
    text='10', angle=0, color='#B3B3B3'
).encode(
    x='x:T', y='y:Q'
)

rule_P10= alt.Chart({'values':[{'y': -1.281551566}]}).mark_rule(color='#B3B3B3').encode(
    y='y:Q',
)

text_P99 = alt.Chart({'values':[{'x': '2021-02-01', 'y': 2.326347874}]}).mark_text(
    text='99', angle=0, color='#B3B3B3'
).encode(
    x='x:T', y='y:Q'
)



rule_P99= alt.Chart({'values':[{'y': 2.326347874}]}).mark_rule(color='#B3B3B3').encode(
    y='y:Q',
)

rule_P999= alt.Chart({'values':[{'y': 3.090232306}]}).mark_rule(color='#B3B3B3').encode(
    y='y:Q',
)

text_P999 = alt.Chart({'values':[{'x': '2021-02-01', 'y': 3.090232306}]}).mark_text(
    text='99.9', angle=0, color='#B3B3B3'
).encode(
    x='x:T', y='y:Q'
)

text_P05 = alt.Chart({'values':[{'x': '2021-02-01', 'y': -1.644853627}]}).mark_text(
    text='5', angle=0, color='#B3B3B3'
).encode(
    x='x:T', y='y:Q'
)

rule_P05= alt.Chart({'values':[{'y': -1.644853627}]}).mark_rule(color='#B3B3B3').encode(
    y='y:Q',
)

rule_P9999= alt.Chart({'values':[{'y': 3.719016485}]}).mark_rule(color='#B3B3B3').encode(
    y='y:Q',
)

text_P9999 = alt.Chart({'values':[{'x': '2021-02-01', 'y': 3.719016485}]}).mark_text(
    text='99.99', angle=0, color='#B3B3B3'
).encode(
    x='x:T', y='y:Q'
)



(feb_chart+wind_problem+near_grid_collapse+rule_wind+rule_collapse+
 rule_P50+text_P50+rule_P90+text_P90+rule_P10+text_P10+rule_P99+text_P99+rule_P999+text_P999+rule_P05+text_P05+
    text_P9999+rule_P9999).properties(
    title=['February ERCOT Power Probabilities']
)

alt.LayerChart(...)

# Demand Graphing

The next set of cells are used to generate graphs related to forecast demand and the expected loads according to ERCOT's SARA analysis

In [12]:
eia_df_2 = pd.read_csv('assets/930-data-export_2.csv').dropna()
central = pytz.timezone('America/Chicago')
eia_df_2['Timestamp (Hour Ending)'] = eia_df_2['Timestamp (Hour Ending)'].replace({'CST': 'UTC+6'}, regex=True)
eia_df_2['datetime']  = pd.to_datetime(eia_df_2['Timestamp (Hour Ending)'], utc=True).dt.tz_convert(central)
eia_df_2 = eia_df_2.rename(columns={
    'Demand (MWh)':'demand',
    'Demand Forecast (MWh)':'demand_forecast',
    'Net Generation (MWh)':'net_generation',
    'Total Interchange (MWh)':'total_interchange'
})

In [13]:
demand_chart = alt.Chart(eia_df_2).mark_line().encode(
    x=alt.X('datetime:T',
         axis=alt.Axis(title='Date')
     ),
    y=alt.Y('demand_forecast:Q',
         axis=alt.Axis(title='Demand Forecast, MWh')
     )
)

text_peak = alt.Chart({'values':[{'x': '2021-02-07', 'y': 56000}]}).mark_text(
    text='SARA Peak Demand Forecast', angle=0, color='orange'
).encode(
    x='x:T', y='y:Q'
)

rule_peak= alt.Chart({'values':[{'y': 57699}]}).mark_rule(color='orange').encode(
    y='y:Q',
)

text_extreme = alt.Chart({'values':[{'x': '2021-02-07', 'y': 65500}]}).mark_text(
    text='SARA Extreme Load Forecast', angle=0, color='red'
).encode(
    x='x:T', y='y:Q'
)

rule_extreme= alt.Chart({'values':[{'y': 67208}]}).mark_rule(color='red').encode(
    y='y:Q',
)

(demand_chart+text_peak+rule_peak+text_extreme+rule_extreme).properties(
    title=['February ERCOT Demand Forecast with SARA Peak and Extreme Load']
)

alt.LayerChart(...)

In [14]:
#Converting demand forecast to probability space
demand_mean = 57699
demand_std = (67208-demand_mean)/st.norm.ppf(.90)

eia_df_2['z_score'] = (eia_df_2['demand_forecast']-demand_mean)/demand_std

In [15]:

plot_eia_df_2 = eia_df_2[eia_df_2['datetime']>'2021-02-01 06:00:00-06:00']
#print(plot_eia_df_2)

demand_prob_chart = alt.Chart(plot_eia_df_2).mark_line().encode(
    x=alt.X('datetime:T',
         axis=alt.Axis(title='Date')
     ),
    y=alt.Y('z_score:Q',
         axis=alt.Axis(title='Probabiltiy of Peak Demand Less Than, %', labels=False, grid=False)
     )
)

(demand_prob_chart+
 rule_P50+text_P50+rule_P90+text_P90+rule_P10+text_P10+rule_P99+text_P99+rule_P05+text_P05
    ).properties(
    title=['February ERCOT Peak Demand Probabilities']
)

alt.LayerChart(...)

# Supply Graphing

The next set of cells are used to generate graphs related to forecast supply and the expected loads according to ERCOT's SARA analysis and six day forcast supply

In [16]:
ercot_02_09 = pd.read_csv('assets/2021_02_09.csv')
ercot_02_09['forecast_date'] = date(2021,2,9)

ercot_02_10 = pd.read_csv('assets/2021_02_10.csv')
ercot_02_10['forecast_date'] = date(2021,2,10)

ercot_02_11 = pd.read_csv('assets/2021_02_11.csv')
ercot_02_11['forecast_date'] = date(2021,2,11)

ercot_02_12 = pd.read_csv('assets/2021_02_12.csv')
ercot_02_12['forecast_date'] = date(2021,2,12)

ercot_02_13 = pd.read_csv('assets/2021_02_13.csv')
ercot_02_13['forecast_date'] = date(2021,2,13)

ercot_02_14 = pd.read_csv('assets/2021_02_14.csv')
ercot_02_14['forecast_date'] = date(2021,2,14)

ercot_forecast = pd.concat([ercot_02_09,ercot_02_10,ercot_02_11,ercot_02_12,ercot_02_13,ercot_02_14])
ercot_forecast['hour'] = pd.to_timedelta(pd.to_numeric(ercot_forecast['HourEnding'].str[0:2])-1,unit='h')
ercot_forecast['datetime'] = pd.to_datetime(ercot_forecast['DeliveryDate']).dt.tz_localize('America/Chicago')

ercot_forecast['datetime'] = ercot_forecast['datetime']+ercot_forecast['hour']
ercot_forecast['total_supply'] = ercot_forecast['TotalCapGenRes']+ercot_forecast['OfflineAvailableMW']
ercot_forecast = ercot_forecast[ercot_forecast['OfflineAvailableMW']>0]
ercot_forecast = ercot_forecast.drop(columns=['DSTFlag','DeliveryDate','HourEnding','hour','TotalCapGenRes','TotalCapLoadRes','OfflineAvailableMW'])
ercot_forecast['forecast_date'] = pd.to_datetime(ercot_forecast['forecast_date'])

In [17]:
domain_pd = pd.to_datetime(['2021-02-09', '2021-02-21']).astype(int) / 10 ** 6

supply_chart = alt.Chart(ercot_forecast).mark_line(color='#3f8a8d').encode(
    x=alt.X('datetime:T',
        axis=alt.Axis(title='Date'),
        scale=alt.Scale(domain=list(domain_pd))
     ),
    y=alt.Y('total_supply:Q',
         axis=alt.Axis(title='Total Available Supply (Generating+Reserve), MWh')
     ),
    color=alt.Color('forecast_date:O', legend=alt.Legend(title="Forecast As Of Date"))
)

actual_chart = alt.Chart(eia_df_2).mark_line(color="#8f4267",clip=True).encode(
    x='datetime:T',
    y='net_generation:Q'
)

# A dropdown filter
forecast_dates = ercot_forecast['forecast_date'].unique()
date_dropdown = alt.binding_select(options=forecast_dates)
date_select = alt.selection_single(fields=['forecast_date'], bind=date_dropdown, name="_")

filter_date = supply_chart.add_selection(
    date_select
).transform_filter(
    date_select
).properties(title="Supply Forecast")


text_max = alt.Chart({'values':[{'x': '2021-02-11', 'y': 81000 }]}).mark_text(
    text='SARA Maximum Supply', angle=0, color='green'
).encode(
    x='x:T', y='y:Q'
)

rule_max= alt.Chart({'values':[{'y': 82513 }]}).mark_rule(color='green').encode(
    y='y:Q',
)


text_average = alt.Chart({'values':[{'x': '2021-02-10 15:00:00:00-06:00', 'y':  72000 }]}).mark_text(
    text='SARA Average Supply', angle=0, color='orange'
).encode(
    x='x:T', y='y:Q'
)

rule_average= alt.Chart({'values':[{'y': 73897}]}).mark_rule(color='orange').encode(
    y='y:Q',
)

text_min = alt.Chart({'values':[{'x': '2021-02-11', 'y': 67000}]}).mark_text(
    text='SARA Minimum Supply', angle=0, color='red'
).encode(
    x='x:T', y='y:Q'
)

rule_min= alt.Chart({'values':[{'y': 68560}]}).mark_rule(color='red').encode(
    y='y:Q',
)

text_actual = alt.Chart({'values':[{'x': '2021-02-13', 'y': 35000}]}).mark_text(
    text='Actual Supply (Generating Only)', angle=0, color='#8f4267'
).encode(
    x='x:T', y='y:Q'
)


filter_date+actual_chart+text_max+rule_max+text_average+rule_average+text_min +rule_min+text_actual

alt.LayerChart(...)